# **Algoritmo de Regresión: Perceptrón Paralelo**

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler



Extracción de datos: dataset en https://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength



In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls"
X = pd.read_excel(url)
y = X.iloc[:,-1] # la variable independiente se encuentra en la ultima columna
X.drop(X.columns[-1], axis='columns', inplace=True)
X.columns = range(X.shape[1])
print(X)
print(y)

          0      1      2      3     4       5      6    7
0     540.0    0.0    0.0  162.0   2.5  1040.0  676.0   28
1     540.0    0.0    0.0  162.0   2.5  1055.0  676.0   28
2     332.5  142.5    0.0  228.0   0.0   932.0  594.0  270
3     332.5  142.5    0.0  228.0   0.0   932.0  594.0  365
4     198.6  132.4    0.0  192.0   0.0   978.4  825.5  360
...     ...    ...    ...    ...   ...     ...    ...  ...
1025  276.4  116.0   90.3  179.6   8.9   870.1  768.3   28
1026  322.2    0.0  115.6  196.0  10.4   817.9  813.4   28
1027  148.5  139.4  108.6  192.7   6.1   892.4  780.0   28
1028  159.1  186.7    0.0  175.6  11.3   989.6  788.9   28
1029  260.9  100.5   78.3  200.6   8.6   864.5  761.5   28

[1030 rows x 8 columns]
0       79.986111
1       61.887366
2       40.269535
3       41.052780
4       44.296075
          ...    
1025    44.284354
1026    31.178794
1027    23.696601
1028    32.768036
1029    32.401235
Name: Concrete compressive strength(MPa, megapascals) , Length: 1030,

Defina la proporción de datos para el conjunto de entrenamiento

In [ ]:
prop = float(input("Proporcion de datos para el conjunto de entrenamiento: p = ")) # proporcion del conjunto de entrenamiento
X_train_o, X_test_o, y_train_o, y_test_o = train_test_split(X, y, train_size=prop) #, random_state=0)

# Juntamos X_train_o e y_train_o en el mismo dataframe
Xny_train_o = pd.DataFrame(X_train_o) 
d = Xny_train_o.shape[1] 
Xny_train_o[d] = pd.Series(y_train_o)

# Juntamos X_test_o e y_test_o en el mismo dataframe
Xny_test_o = pd.DataFrame(X_test_o) 
Xny_test_o[d] = pd.Series(y_test_o)


Proporcion de datos para el conjunto de entrenamiento: p = 0.3


In [3]:
prop = float(input("Proporcion de datos para el conjunto de entrenamiento: p = ")) # proporcion del conjunto de entrenamiento
X_train_o, X_test_o, y_train_o, y_test_o = train_test_split(X, y, train_size=prop) #, random_state=0)

Proporcion de datos para el conjunto de entrenamiento: p = 0.6


Debemos escalar los vectores de características y extender los vectores (añadir el componente para el bias).

In [ ]:
scaler = MinMaxScaler((-1,1))
Xny_train = scaler.fit_transform(Xny_train_o)
Xny_test = scaler.transform(Xny_test_o)

Xny_train = pd.DataFrame(Xny_train)
Xny_test = pd.DataFrame(Xny_test)

y_train = Xny_train.iloc[:,-1] # la variable independiente se encuentra en la ultima columna
X_train = Xny_train.drop(Xny_train.columns[-1], axis='columns')
X_train.columns = range(X_train.shape[1])

y_test = Xny_test.iloc[:,-1] # la variable independiente se encuentra en la ultima columna
X_test = Xny_test.drop(Xny_test.columns[-1], axis='columns')
X_test.columns = range(X_test.shape[1])

d = X_train.shape[1]

bias = [1 for i in range(X_train.shape[0])] 
X_train[d] = pd.Series(bias) # añadir componente de bias al conjunto de entrenamiento

bias = [1 for i in range(X_test.shape[0])]
X_test[d] = pd.Series(bias) # añadir componente de bias al conjunto de prueba

d = d+1 # dimensión del vector de características extendido

In [ ]:
print(X_train)
print(y_train)
print(X_test)
print(y_test)

            0         1       2         3  ...         5         6         7  8
0    0.814686 -1.000000 -1.0000  0.472941  ...  0.883721 -0.904666  0.478022  1
1   -0.619921 -1.000000  0.2518 -0.155576  ...  0.616279  0.028098 -0.456044  1
2   -0.428075 -1.000000  0.2162 -0.017224  ...  0.484884 -0.074511 -0.456044  1
3   -0.218207 -1.000000 -0.0261 -0.268706  ...  0.005233  0.412995 -0.928571  1
4   -0.658374 -0.703832  0.2197 -0.087812  ...  0.502326 -0.063472 -0.851648  1
..        ...       ...     ...       ...  ...       ...       ...       ... ..
304  1.000000 -1.000000 -1.0000 -0.035294  ...  0.883721 -0.904666 -0.928571  1
305 -0.343526 -1.000000 -0.0431  0.236141  ... -0.093837  0.340542 -0.851648  1
306 -0.401436  0.555847 -1.0000  1.000000  ... -0.238372 -1.000000 -0.510989  1
307  0.291174 -0.868981 -0.0600 -0.336471  ... -0.203488  0.259408 -0.851648  1
308 -0.774380  0.546020 -1.0000  0.322353  ... -0.218605 -0.060712 -0.989011  1

[309 rows x 9 columns]
0      0.339714


### Distancia Minkowski de $x,y$ vectores en $\mathbb{R}^n$





In [ ]:
def MinkowskiDistance(x,y, p=2): 
  dist = 0
  n = len(x)

  for i in range(0,n):
    dist += abs(x[i]-y[i])**p

  dist = dist**(1/p)

  return dist

### Clase del perceptrón

In [ ]:
class perceptron:
  def __init__(self, d):
    self.d = d # d es la dimension de los vectores de caracteristicas extendidos
    self.alpha = np.random.uniform(-0.5, 0.5, d) # El vector de pesos se inicializa con valores aleatorios en [-0.5, 0.5]

  def decision_function(self, z):
    # z es el vector de caracteristicas extendido
    # funcion de decision f
    # f(z) = 1, si (alpha)(z) >= 0 (producto punto de alpha y z), y f(z) = -1 en otro caso 

    if np.dot(self.alpha, z)  >= 0:
      return 1
    else:
      return -1

  def normalize_alpha(self):
    # función para normalizar el vector de pesos alpha

    norm = MinkowskiDistance(self.alpha, np.zeros(self.d)) # la norma de alpha es la distancia euclidiana de alpha al vector 0

    self.alpha = (1/norm)*self.alpha



###Clase del perceptrón paralelo

In [ ]:
class parallel_perceptron:
  def __init__(self, d, n, margin, epsilon, mu, rho):
    self.d = d # d es la dimension de los vectores de caracteristicas extendidos
    self.n = n # numero de perceptrones en la capa
    self.margin = margin # margen
    self.epsilon = epsilon # exactitud del algoritmo
    self.mu = mu # importancia de un margen claro
    self.rho = rho # resolution of squashing function

    self.capa = [perceptron(d) for i in range(n)]

  def squash_function(self, z):
    # z es el vector de caracteristicas extendido
    # funcion s(p) = -1 si p <= -rho, s(p)= p/rho si -rho < p < rho, y s(p) = 1 si rho <= p
    # donde p = f_1(z)+...+f_n(z) y f_i es la funcion de decision del perceptron i

    p = 0

    for i in range(self.n):
      p+= self.capa[i].decision_function(z)

    if p >= self.rho:
      return 1
    elif p<= -self.rho:
      return -1
    else:
      return float(p)/float(self.rho)

  def p_deltaRule(self, z, o, t):
    # z es el vector de caracteristicas extendido
    # o es la clase del vector z
    # Regla de actualización de los perceptrones en la época t

    eta = 1/(2*(t**(0.5)))

    o_h = self.squash_function(z) # prediccion de o

    for i in range(self.n):
    
      dot_prod = np.dot(self.capa[i].alpha, z)

      if o_h > o + self.epsilon and dot_prod >= 0:
        result = -z
      elif o_h < o-self.epsilon and dot_prod < 0:
        result = z
      elif o_h <= o+self.epsilon and 0 <= dot_prod < self.margin:
        result = self.mu * z
      elif o_h >= o-self.epsilon and -self.margin < dot_prod < 0: 
        result = -self.mu * z
      else:
        result = 0

      self.capa[i].alpha = self.capa[i].alpha + eta*result

      self.capa[i].normalize_alpha()

  def fit(self, X_train, y_train, num_epochs):
    # Función de entrenamiento del perceptrón paralelo usando la regla p-delta
    X_train = X_train.to_numpy()
    y_train = y_train.to_numpy()

    N = X_train.shape[0]

    # Por cada época t, entrenamos el perceptrón paralelo con una permutación aleatoria de los datos

    for t in range(1,num_epochs+1):
      print(t)
      #ind = np.random.choice(N, N, replace=False) # Permutación aleatoria de los números 0, 1, ..., N-1

      # Regla p-delta para cada elemento del conjunto de entrenamiento
      for i in range(N):
        self.p_deltaRule(X_train[i], y_train[i], t)

  def predict(self, X_test):
    X_test = X_test.to_numpy()
    N = X_test.shape[0]

    labels = []

    for i in range(N):
      labels.append(self.squash_function(X_test[i]))

    return pd.Series(labels)



### Función que calcula el error cuadrático medio (mean-squared error):
$$
MSE(y, \hat{y})=\frac{1}{n_{samples}}\sum_{i=0}^{n_{samples}-1} (y_i-\hat{y}_i)^2.
$$

In [ ]:
def MSE(pred, real):

  pred = pred.to_numpy()
  real = real.to_numpy()

  n = len(real)

  mse = 0

  for i in range(n):
    mse += (pred[i]-real[i])**2

  mse = mse/n

  return mse

Parámetros del perceptrón paralelo

In [ ]:
n = int(input("Numero de perceptrones: n = "))
margin = float(input("Margen (margin) = "))
epsilon = float(input("Exactitud (epsilon) = "))
mu = float(input("importancia del margen claro (mu) = "))
num_epochs = int(input("Numero de epocas = "))
rho = float(input("Resolucion de la squashing function = "))

In [ ]:
n = 5000
margin = 0.05
epsilon = 0.1
mu = 1
num_epochs = 5
rho = 5000

Clasificación del conjunto de entrenamiento. Impresión de las etiquetas predichas, matriz de confunsión y exactitud.



In [ ]:
regressor = parallel_perceptron(d, n, margin, epsilon, mu, rho)
regressor.fit(X_train, y_train, num_epochs)
pred = regressor.predict(X_train)

1
2
3
4
5


In [ ]:
mse_train = MSE(pred, y_train)
print(pred)
print(y_train)
print(mse_train)

0     -1
1      1
2      1
3      1
4      1
      ..
304   -1
305    1
306   -1
307    1
308    1
Length: 309, dtype: int64
0      0.339714
1     -0.234575
2      0.091809
3     -0.013257
4     -0.295634
         ...   
304    0.462200
305   -0.350093
306   -0.246457
307    0.208794
308   -0.869355
Name: 8, Length: 309, dtype: float64
1.610932371616562


In [ ]:
for i in range(len(pred)):
  print(pred[i])

-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
1
1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
-1
1
1
1
1
1
1
1
1
1
-1
1
1
1
1
1
1
1
1
1
-1
1
1
1
1
1
1
1
1
1
1
1
-1
1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
1
1
1
1
1
1
1
1
-1
1
1
1
-1
1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
1
1
1
1
1
1
1
1
-1
1
1
1
1
1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
1
1
-1
1
1
1
1
1
1
1
1
1
-1
1
1
1
1
1
1
1
1
1
1
1
-1
1
1
-1
1
1
-1
1
-1
1
-1
1
1


Clasificación del conjunto de prueba. Impresión de las etiquetas predichas, matriz de confunsión y exactitud.

In [ ]:
pred_test = regressor.predict(X_test)

In [ ]:
mse_test = MSE(pred_test, y_test)
print(pred_test)
print(y_test)
print(mse_test)

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
407    1.0
408    1.0
409    1.0
410    1.0
411    1.0
Length: 412, dtype: float64
0     -0.267363
1     -0.776702
2     -0.308453
3      0.055624
4     -0.293267
         ...   
407   -0.203584
408    0.444033
409   -0.089603
410   -0.503184
411   -0.592868
Name: 8, Length: 412, dtype: float64
1.5812046451204351
